In [ ]:
import torch.nn as nn
import torch.nn.functional as f
import torch
import numpy as np
from torchvision import transforms,datasets
import torchvision
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

In [ ]:
!nvidia-smi

In [ ]:
folders = glob("../input/face-expression-recognition-dataset/images/train/*")
len(folders)

In [ ]:
folders

In [ ]:
class modification(nn.Module):
    def __init__(self):
        super(modification,self).__init__()
    def forward(self,x):
        return x

In [ ]:
model = torchvision.models.vgg16(pretrained = True)
model.avgpool = modification()
model.classifier[6] = nn.Linear(4096,7)
print(model)

In [ ]:
model.to(device)

In [ ]:
#Hyperparameters
batch_size = 64
img_size = 224
learning_rate = 1e-4

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize(img_size),transforms.CenterCrop(img_size),transforms.RandomHorizontalFlip()])
dir_path = '../input/face-expression-recognition-dataset/images'
train_path = dir_path + '/train'
val_path = dir_path + '/validation'
dataset = datasets.ImageFolder(train_path,transform)
train_dataloader = torch.utils.data.DataLoader(dataset,batch_size = batch_size,shuffle = True)
validation_dataset = datasets.ImageFolder(val_path,transform)
val_dataloader = torch.utils.data.DataLoader(validation_dataset,batch_size = batch_size,shuffle = True)

In [ ]:
#display data
images,labels = next(iter(train_dataloader))
for i in range(2):
    plt.imshow(images[i].view(224,224,3).numpy())
    plt.title(labels[i])
    plt.show()
# plt.imshow(images[0])
print(labels[:5])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
def train(model,criterion,train_dataloader,optimiser,epochs = 10):
    model.train()
    for epoch in range(epochs):
        loop = tqdm(enumerate(train_dataloader),total = len(train_dataloader),leave = False)
        for batch_index,(x,y) in loop:
            x = x.to(device = device)
            y = y.to(device = device)
            optimiser.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred,y)
            loss.backward()
            optimiser.step
            loop.set_description(f"Epoch [{epoch} out of {epochs}]")
            loop.set_postfix(Loss = loss.item())                      
    print('Training completed')
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader,leave = False):
            x = x.to(device = device)
            y = y.to(device = device)
            y_pred = model(x)
            _,predictions = y_pred.max(1)
            num_correct += (predictions)
            num_samples += predictions.size(0)
        accuracy = (float(num_correct)/float(num_samples))*100
    model.train()
    return accuracy

In [ ]:
train(model,criterion,train_dataloader,optimiser,epochs = 10)
print('Training Accuracy:',check_accuracy(train_dataloader,model))
print('Validation Accuracy:',check_accuracy(val_dataloader,model))